# Italian Literature-based queries

In [1]:
from pandas import *
from queryWikidata import query_wikidata

In [2]:
endpoint = "https://query.wikidata.org/sparql"
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:105.0) Gecko/20100101 Firefox/105.0"

## `writers_and_works`

#### It retrieves all italian authors of written works having one or more work linked on wikidata

In [3]:
writers_and_works_query = """
SELECT DISTINCT ?person ?personLabel ?authorImdbId
(COUNT(DISTINCT ?work) AS ?work)
(COUNT(DISTINCT ?workCreated) AS ?creatorP) (COUNT(DISTINCT ?workAuthored) AS ?authorP) (COUNT(DISTINCT ?workInAuthorsLists) AS ?workInAuthorsLists)
(COUNT(DISTINCT ?occupationWriter) AS ?writer) (COUNT(DISTINCT ?occupationNovelist) AS ?novelist) (COUNT(DISTINCT ?occupationPoet) AS ?poet) (COUNT(DISTINCT ?occupationProsaist) AS ?prosaist) (COUNT(DISTINCT ?occupationScreenwriter) AS ?screenwriter)

WHERE {
    {
      ?person ?occupationWriter  wd:Q36180.
    }UNION{
      ?person ?occupationNovelist  wd:Q6625963
    }UNION{
      ?person ?occupationPoet  wd:Q49757
    }UNION{
      ?person ?occupationProsaist  wd:Q12144794
    }UNION{
      ?person ?occupationScreenwriter wd:Q28389
    }
    

    ?person wdt:P31 wd:Q5 ;
            wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  
    ?capitalCity wdt:P31 wd:Q515 ; #get cities
                  wdt:P17 wd:Q38 .


    {
      ?work wdt:P170 ?person .
      BIND (?work AS ?workCreated)
     }UNION{
      ?work wdt:P50 ?person .                            
      BIND (?work AS ?workAuthored)
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?person ?authorOf ?work .
      BIND (?work AS ?workInAuthorsLists)
    }
    
    {                                                  #more efficient
      ?work wdt:P31 wd:Q47461344 .
    }UNION{
      ?work wdt:P31 wd:Q7725634 .
    }UNION{
      ?work wdt:P31 wd:Q571 .
    }UNION{
      ?work wdt:P31 wd:Q3331189 .
    }
    
    #{                                                    #less efficient
    #  VALUES ?writtenWork {wd:Q571 wd:Q3331189}          #VALUES ?writtenWork {wd:Q47461344 wd:Q571 wd:Q3331189 wd:Q7725634 } . ?work wdt:P31 ?writtenWork  non funziona
    #  ?work wdt:P31 ?writtenWork .
    #}UNION{
    #  VALUES ?writtenWork {wd:Q47461344 wd:Q7725634}
    #  ?work wdt:P31 ?writtenWork .
    #}
  
  
  OPTIONAL {?person wdt:P345 ?authorImdbId}
 
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?person ?personLabel ?authorImdbId
"""

ww_df = query_wikidata(endpoint, writers_and_works_query, user_agent, True).fillna("")
ww_df

KeyboardInterrupt: 

In [ ]:
ww_df.sum()

person                http://www.wikidata.org/entity/Q25163http://ww...
personLabel           Roberto SavianoPaolo TerniLuisito BianchiCesar...
authorImdbId          nm2500207nm0136346nm3842362nm0110292nm1202191n...
work                                                               5298
creatorP                                                             75
authorP                                                            5229
workInAuthorsLists                                                  409
writer                                                             1212
novelist                                                             72
poet                                                                320
prosaist                                                             14
screenwriter                                                        249
dtype: object

## `writers_works_and_derivatives`

#### It retrieves all italian authors of written works that are linked to one of their work on wikidata, which are in turn linked to some derivative work on wikidata

In [6]:
ww_and_derivatives_query = """
SELECT DISTINCT ?person ?personLabel ?authorImdbId
(COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWork)
(COUNT(DISTINCT ?influencedWork) AS ?influencedByP) (COUNT(DISTINCT ?workBasedOn) AS ?basedOnP) (COUNT(DISTINCT ?referencedWork) AS ?referencesWorkP) (COUNT(DISTINCT ?derivedWork) AS ?derivedWorkP) (COUNT(DISTINCT ?inspiredWork) AS ?inspiredByP)  (COUNT(DISTINCT ?workAfter) AS ?afterWorkByP)

WHERE {
    
    {
      ?person wdt:P106  wd:Q36180.
    }UNION{
      ?person wdt:P106  wd:Q6625963
    }UNION{
      ?person wdt:P106  wd:Q49757
    }UNION{
      ?person wdt:P106  wd:Q12144794
    }UNION{
      ?person wdt:P106 wd:Q28389
    }
    
    
    ?person wdt:P31 wd:Q5 ;
            wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  
    ?capitalCity wdt:P31 wd:Q515 ; #get cities
                  wdt:P17 wd:Q38 .


    {
      ?work wdt:P170 ?person .
     }UNION{
      ?work wdt:P50 ?person .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?person ?authorOf ?work .
    }
    
    {                                                  #more efficient
      ?work wdt:P31 wd:Q47461344 .
    }UNION{
      ?work wdt:P31 wd:Q7725634 .
    }UNION{
      ?work wdt:P31 wd:Q571 .
    }UNION{
      ?work wdt:P31 wd:Q3331189 .
    }
    
    #{                                                    #less efficient
    #  VALUES ?writtenWork {wd:Q571 wd:Q3331189}          #VALUES ?writtenWork {wd:Q47461344 wd:Q571 wd:Q3331189 wd:Q7725634 } . ?work wdt:P31 ?writtenWork does not work
    #  ?work wdt:P31 ?writtenWork .
    #}UNION{
    #  VALUES ?writtenWork {wd:Q47461344 wd:Q7725634}
    #  ?work wdt:P31 ?writtenWork .
    #}


    {
      ?derivativeWork wdt:P737 ?work . 
      BIND(?derivativeWork AS ?influencedWork)
    }UNION{
      ?derivativeWork wdt:P144 ?work .
      BIND(?derivativeWork AS ?workBasedOn)
    }UNION{
      ?derivativeWork wdt:P941 ?work .
      BIND(?derivativeWork AS ?inspiredWork)
    }UNION{  
      ?derivativeWork wdt:P8371 ?work .
      BIND(?derivativeWork AS ?referencedWork)
    }UNION{
      ?work wdt:P4969 ?derivativeWork .
      BIND(?derivativeWork AS ?derivedWork)
    }UNION{
      ?derivativeWork wdt:P1877 ?person .
      BIND(?derivativeWork AS ?workAfter)
    }
    
  
  OPTIONAL {?person wdt:P345 ?authorImdbId}
 
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?person ?personLabel ?authorImdbId
"""

wwd_df = query_wikidata(endpoint, ww_and_derivatives_query, user_agent, True).fillna("")
wwd_df

,person,personLabel,authorImdbId,work,derivativeWork,influencedByP,basedOnP,referencesWorkP,derivedWorkP,inspiredByP,afterWorkByP
0,http://www.wikidata.org/entity/Q295589,Alessandro Baricco,nm0054621,20,13,0,5,0,3,0,10
1,http://www.wikidata.org/entity/Q741862,Massimo Bontempelli,nm0095219,1,1,0,1,0,0,0,0
2,http://www.wikidata.org/entity/Q3680870,Clizia Gurrado,,1,1,0,1,0,1,0,0
3,http://www.wikidata.org/entity/Q368127,Elena Ferrante,nm0273919,12,4,0,4,0,4,0,1
4,http://www.wikidata.org/entity/Q25120,Pier Paolo Pasolini,nm0001596,18,2,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...
117,http://www.wikidata.org/entity/Q334188,Pietro Bembo,,1,1,0,1,0,0,0,0
118,http://www.wikidata.org/entity/Q7371,Federico Fellini,nm0000019,2,2,0,0,0,0,0,2
119,http://www.wikidata.org/entity/Q3827806,Laura Mancinelli,,2,8,0,7,0,5,1,0
120,http://www.wikidata.org/entity/Q469656,Paolo Giordano,nm1279468,1,1,0,1,0,0,0,0


In [7]:
wwd_df.sum()

person             http://www.wikidata.org/entity/Q295589http://w...
personLabel        Alessandro BariccoMassimo BontempelliClizia Gu...
authorImdbId       nm0054621nm0095219nm0273919nm0001596nm0685063n...
work                                                            1269
derivativeWork                                                   456
influencedByP                                                      5
basedOnP                                                         335
referencesWorkP                                                    3
derivedWorkP                                                     132
inspiredByP                                                       23
afterWorkByP                                                     146
dtype: object

## `derivatives_type`

#### Retrieves the type of derivative works connected with an author's works

In [16]:
derivatives_type = """
SELECT ?derivativeWorkClassLabel (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWork) 
WHERE {
    
    {
      ?person wdt:P106  wd:Q36180
    }UNION{
      ?person wdt:P106  wd:Q6625963
    }UNION{
      ?person wdt:P106  wd:Q49757
    }UNION{
      ?person wdt:P106  wd:Q12144794
    }UNION{
      ?person wdt:P106 wd:Q28389 
    }
    

    ?person wdt:P31 wd:Q5 ;
            wdt:P27 ?country .                          #the block gets the writers having had the citizenship of a country, whose capital city is now an Italian city
    ?country (wdt:P36 | p:P36 / ps:P36) ?capitalCity .  
    ?capitalCity wdt:P31 wd:Q515 ; #get cities
                  wdt:P17 wd:Q38 .


    {
      ?work wdt:P170 ?person .
     }UNION{
      ?work wdt:P50 ?person .                       
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?person ?authorOf ?work .
    }
    
    {                                                
      ?work wdt:P31 wd:Q47461344 .
    }UNION{
      ?work wdt:P31 wd:Q7725634 .
    }UNION{
      ?work wdt:P31 wd:Q571 .
    }UNION{
      ?work wdt:P31 wd:Q3331189 .
    }
    
    {
      VALUES ?p {wdt:P737 wdt:P144 wdt:P941 wdt:P8371 wdt:P4969}   
      ?derivativeWork ?p ?work .
    }UNION{
      ?derivativeWork wdt:P1877 ?person .
    }
   
    ?derivativeWork wdt:P31 ?derivativeWorkClass .
    ?derivativeWorkClass rdfs:label ?derivativeWorkClassLabel .
    FILTER((LANG(?derivativeWorkClassLabel)) = "en")
                              
} 
GROUP BY ?derivativeWorkClassLabel 
"""

dt_df = query_wikidata(endpoint, derivatives_type, user_agent, True).fillna("")
dt_df.sort_values(by="derivativeWork", ascending=False)

,derivativeWorkClassLabel,derivativeWork
0,film,143
2,literary work,53
4,theatrical production,49
3,dramatico-musical work,42
5,operatic production,22
...,...,...
19,dance production,1
39,film project,1
40,stop-motion animated film,1
17,cycle of paintings,1


In [17]:
dt_df.sum()

derivativeWorkClassLabel    filmtelevision filmliterary workdramatico-musi...
derivativeWork                                                            490
dtype: object

In [20]:
others = 0
classes = []
count = []
for idx, row in dt_df.iterrows():
    if row["derivativeWork"] < 5:
        others += row["derivativeWork"]
    else:
        classes.append(row["derivativeWorkClassLabel"])
        count.append(row["derivativeWork"])

classes.append("Others")
count.append(others)

simplified_dt_df = DataFrame({"derivativeWorkClassLabel": classes, "derivativeWork": count})
simplified_dt_df

,derivativeWorkClassLabel,derivativeWork
0,film,143
1,television film,12
2,literary work,53
3,dramatico-musical work,42
4,theatrical production,49
5,operatic production,22
6,teleplay,12
7,painting,18
8,musical work/composition,10
9,miniseries,14
